# import など準備

In [1]:
import numpy as np
import pandas as pd
from retry import retry
import sys
import os
from robobrowser import RoboBrowser

In [32]:
# 比較参照用
kabupro = pd.read_csv('/Users/Really/Stockyard/_csv/kessan_table.csv')

In [48]:
kabupro.ix[14]
# 株プロにしか無い項目: 希薄化後一株当り純利益, 純資産又は株主資本, 営業キャッシュフロー, 投資キャッシュフロー, 財務キャッシュフロー

証券コード                      1301
企業名                          極洋
会計基準                       日本基準
連結個別                         連結
決算期                    2017年3月期
決算期間                         通期
期首                   2016-04-01
期末                   2017-03-31
名寄前勘定科目（売上高欄に掲載）            売上高
売上高                 2.36561e+11
営業利益                  3.723e+09
経常利益                  3.709e+09
純利益                   2.422e+09
一株当り純利益                  230.66
希薄化後一株当り純利益              213.01
純資産又は株主資本            2.5391e+10
総資産                  9.7391e+10
一株当り純資産                 2378.09
営業キャッシュフロー             6.01e+08
投資キャッシュフロー           -1.998e+09
財務キャッシュフロー             1.05e+08
情報公開又は更新日            2017-05-11
Name: 14, dtype: object

## 参考コード

In [ ]:
def get_price_yahoojp(code, start=None, end=None, interval='d'): # start = '2017-01-01'
    # http://sinhrks.hatenablog.com/entry/2015/02/04/002258
    # http://jbclub.xii.jp/?p=598
    base = 'http://info.finance.yahoo.co.jp/history/?code={0}.T&{1}&{2}&tm={3}&p={4}'
    
    start = pd.to_datetime(start) # Timestamp('2017-01-01 00:00:00')

    if end == None:
        end = pd.to_datetime(pd.datetime.now())
    else :
        end = pd.to_datetime(end)
    start = 'sy={0}&sm={1}&sd={2}'.format(start.year, start.month, start.day) # 'sy=2017&sm=1&sd=1'
    end = 'ey={0}&em={1}&ed={2}'.format(end.year, end.month, end.day)
    p = 1
    tmp_result = []

    if interval not in ['d', 'w', 'm', 'v']:
        raise ValueError("Invalid interval: valid values are 'd', 'w', 'm' and 'v'")

    while True:
        url = base.format(code, start, end, interval, p)
        # print(url)
        # https://info.finance.yahoo.co.jp/history/?code=7203.T&sy=2000&sm=1&sd=1&ey=2017&em=10&ed=13&tm=d&p=1
        tables = get_table(url)
        if len(tables) < 2 or len(tables[1]) == 0:
            # print('break')
            break
        tmp_result.append(tables[1]) # ページ内の3つのテーブルのうち2番目のテーブルを連結
        p += 1
        # print(p)
        
    result = pd.concat(tmp_result, ignore_index=True) # インデックスをゼロから振り直す

    result.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'AdjClose'] # 列名を変更
    if interval == 'm':
        result['Date'] = pd.to_datetime(result['Date'], format='%Y年%m月')
    else:
        result['Date'] = pd.to_datetime(result['Date'], format='%Y年%m月%d日') # 日付の表記を変更
    result = result.set_index('Date') # インデックスを日付に変更
    result = result.sort_index()
    
    stock_name = tables[0].columns[0]
    # print([code, stock_name])
    
    return [result, stock_name]

In [ ]:
%%writefile amazon_order_history.py

# Amazon.co.jpの注文履歴を取得する

import sys
import os
from robobrowser import RoboBrowser

# 認証の情報は環境変数から取得する
AMAZON_EMAIL = os.environ['AMAZON_EMAIL']
AMAZON_PASSWORD = os.environ['AMAZON_PASSWORD']

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')


def main():
    # 注文履歴のページを開く
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/gp/css/order-history')
    
    # サインインページにリダイレクトされていることを確認する
    assert 'Amazonサインイン' in browser.parsed.title.string
    
    # name="signIn" というサインインフォームを埋める。
    # フォームのname属性の値はブラウザーの開発者ツールで確認できる。
    form = browser.get_form(attrs={'name': 'signIn'})
    form['email'] = AMAZON_EMAIL
    form['password'] = AMAZON_PASSWORD
    
    # フォームを送信する。正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要。
    print('Signing in...', file=sys.stderr)
    browser.submit_form(form, headers={
        'Referer': browser.url,
        'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
    })
    
    # ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
    # print(browser.parsed.prettify())

    # ページャーをたどる。
    while True:
        assert '注文履歴' in browser.parsed.title.string # 注文履歴画面が表示されていることを確認する。
        
        print_order_history() # 注文履歴を表示する。
        
        link_to_next = browser.get_link('次へ') #「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            break #「次へ」のリンクがない場合はループを抜けて終了する。
            
        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next) # 次へ」というリンクをたどる。
        
        
def print_order_history():
    """
    現在のページのすべての注文履歴を表示する
    """
    for line_item in browser.select('.order-info'):
        order = {} # 注文の情報を格納するためのdict
        # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる
        # 注文の情報のすべての列について反復する
        for column in line_item.select('.a-column'):
            label_element = column.select_one('.label')
            value_element = column.select_one('.value')
            # ラベルと値がない列は無視する。
            if label_element and value_element:
                label = label_element.get_text().strip()
                value = value_element.get_text().strip()
                order[label] = value
        print(order['注文日'], order['合計']) # 注文の情報を表示する。
        

if __name__ == '__main__':
    main()

In [ ]:
!forego run python amazon_order_history.py

## get_table 関数 (retry 付き pd.read_html)

In [19]:
@retry(tries=5, delay=1, backoff=2)
def get_table(url):
    result = pd.read_html(url, header=0) # header引数で0行目をヘッダーに指定。データフレーム型
    
    return result

# データの取得 (ログイン状態)

In [49]:
%%writefile kabutan_get_table.py

import sys
import os
from robobrowser import RoboBrowser
from robobrowser import RoboState
import numpy as np
import pandas as pd

# 認証の情報は環境変数から取得する
KT_EMAIL = os.environ['KT_EMAIL']
KT_PASSWORD = os.environ['KT_PASSWORD']

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')

# ログインページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://account.kabutan.jp/login')

# 株探プレミアムページにいることを確認する
assert '株探プレミアム' in browser.parsed.title.string

# name="signIn" というサインインフォームを埋める。
# フォームのname属性の値はブラウザーの開発者ツールで確認できる。
form = browser.get_form(action='/login')
form['session[email]'] = KT_EMAIL
form['session[password]'] = KT_PASSWORD

# フォームを送信する。
# 正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要な場合がある。
print('Signing in...', file=sys.stderr)
browser.submit_form(form)
'''
browser.submit_form(form, headers={
    'Referer': browser.url,
    'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
})
'''

# 個別銘柄の決算ページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://kabutan.jp/stock/finance?code=7203&mode=k')

# 決算ページにいることを確認する
assert '決算' in browser.parsed.title.string

# ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
# print(browser.parsed.prettify())

# print(browser.find())
# print(browser.select('table'))
# print(browser.select('div.fin_fd:nth-child(7) > table:nth-child(1) > tbody:nth-child(1)'))

#kessan = browser.find()
kessan = browser.select('table')
print(kessan)

#result = pd.read_html(kessan, flavor='lxml') #, header=0
#print(result)

#print('success')

'''
def main():
    # 注文履歴のページを開く
    print('Navigating...', file=sys.stderr)
    browser.open('https://www.amazon.co.jp/gp/css/order-history')
    
    # サインインページにリダイレクトされていることを確認する
    assert 'Amazonサインイン' in browser.parsed.title.string
    
    # name="signIn" というサインインフォームを埋める。
    # フォームのname属性の値はブラウザーの開発者ツールで確認できる。
    form = browser.get_form(attrs={'name': 'signIn'})
    form['email'] = KT_EMAIL
    form['password'] = KT_PASSWORD
    
    # フォームを送信する。正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要。
    print('Signing in...', file=sys.stderr)
    browser.submit_form(form, headers={
        'Referer': browser.url,
        'Accept-Language': 'ja,en-US;q=0.7,en;q=0.3',
    })
    
    # ログインに失敗する場合は、次の行のコメントを外してHTMLのソースを確認すると良い。
    # print(browser.parsed.prettify())

    # ページャーをたどる。
    while True:
        assert '注文履歴' in browser.parsed.title.string # 注文履歴画面が表示されていることを確認する。
        
        print_order_history() # 注文履歴を表示する。
        
        link_to_next = browser.get_link('次へ') #「次へ」というテキストを持つリンクを取得する。
        if not link_to_next:
            break #「次へ」のリンクがない場合はループを抜けて終了する。
            
        print('Following link to next page...', file=sys.stderr)
        browser.follow_link(link_to_next) # 次へ」というリンクをたどる。
        
        
def print_order_history():
    """
    現在のページのすべての注文履歴を表示する
    """
    for line_item in browser.select('.order-info'):
        order = {} # 注文の情報を格納するためのdict
        # ページ内のすべての注文履歴について反復する。ブラウザーの開発者ツールでclass属性の値を確認できる
        # 注文の情報のすべての列について反復する
        for column in line_item.select('.a-column'):
            label_element = column.select_one('.label')
            value_element = column.select_one('.value')
            # ラベルと値がない列は無視する。
            if label_element and value_element:
                label = label_element.get_text().strip()
                value = value_element.get_text().strip()
                order[label] = value
        print(order['注文日'], order['合計']) # 注文の情報を表示する。
        

if __name__ == '__main__':
    main()
'''

Overwriting kabutan_get_table.py


In [52]:
kessan = !forego run python kabutan_get_table.py
kessan

In [60]:
# import
import sys
import os
from robobrowser import RoboBrowser
# from robobrowser.browser import RoboState
import numpy as np
import pandas as pd

# 下のセルで入力したメアドとパスワードが出力に表示されてしまうのでそのままでGitHubに上げちゃダメ！

In [ ]:
# sign-in
# 認証の情報
KT_EMAIL = input('Name?')
KT_PASSWORD = input('Password?')

# RoboBrowserオブジェクトを作成する
browser = RoboBrowser(
    parser='html.parser', # Beatiful Soupで使用するパーサーを指定
    # Cookieが使用できないと表示されてログインできない問題を回避するため
    # 通常のブラウザーのUser-Agent(ここではFirefoxのもの)を使う
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:45.0) Gecko/20100101 Firefox/45.0')

# ログインページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://account.kabutan.jp/login')

# 株探プレミアムページにいることを確認する
assert '株探プレミアム' in browser.parsed.title.string

# name="signIn" というサインインフォームを埋める。
# フォームのname属性の値はブラウザーの開発者ツールで確認できる。
form = browser.get_form(action='/login')
form['session[email]'] = KT_EMAIL
form['session[password]'] = KT_PASSWORD

# フォームを送信する。
# 正常にログインするにはRefererヘッダーとAccept-Languageヘッダーが必要な場合がある。
print('Signing in...', file=sys.stderr)
browser.submit_form(form)

print(browser.select('.is-success')[0].text.strip())

In [133]:
# 個別銘柄の決算ページを開く
print('Navigating...', file=sys.stderr)
browser.open('https://kabutan.jp/stock/finance?code=7203&mode=k') # 7203 トヨタ

# 決算ページにいることを確認する
assert '決算' in browser.parsed.title.string

# kessan = browser.select('table')
# print(kessan)

tables = pd.read_html(str(browser.select('table')), header=0)
tables[12]

Navigating...


,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,U,2013.03,12/05/09,NaN,NaN,NaN,NaN,NaN,NaN,22000000.0,1000000.0,1160000.0,760000.0,－
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,U,NaN,12/11/05,↓↑↑↑‐,↓,↑,↑,↑,‐,21300000.0,1050000.0,1180000.0,780000.0,－
3,↓,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,U,NaN,13/02/05,↑↑↑↑‐,↑,↑,↑,↑,‐,21800000.0,1150000.0,1290000.0,860000.0,－
5,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,U,実績,13/05/08,↑↑↑↑‐,↑,↑,↑,↑,‐,22064192.0,1320888.0,1403649.0,962163.0,90
7,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,U,2014.03,13/05/08,NaN,NaN,NaN,NaN,NaN,NaN,23500000.0,1800000.0,1890000.0,1370000.0,－
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
tables[12].columns

Index(['決算期', '修正日', '修正方向', '売上高', '営業益', '経常益', '最終益', '修正配当', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13'],
      dtype='object')

In [159]:
tables[12].columns = ['会計基準', '決算期', '修正日', 
                                   '結合修正方向', '売上高修正方向', '営業益修正方向', '経常益修正方向', '最終益修正方向', '修正配当修正方向', 
                                   '売上高', '営業益', '経常益', '最終益', '修正配当',]

In [161]:
tables[12][tables[12].index % 2 == 0].reset_index(drop=True)

,会計基準,決算期,修正日,結合修正方向,売上高修正方向,営業益修正方向,経常益修正方向,最終益修正方向,修正配当修正方向,売上高,営業益,経常益,最終益,修正配当
0,U,2013.03,12/05/09,NaN,NaN,NaN,NaN,NaN,NaN,22000000.0,1000000.0,1160000.0,760000.0,－
1,U,NaN,12/11/05,↓↑↑↑‐,↓,↑,↑,↑,‐,21300000.0,1050000.0,1180000.0,780000.0,－
2,U,NaN,13/02/05,↑↑↑↑‐,↑,↑,↑,↑,‐,21800000.0,1150000.0,1290000.0,860000.0,－
3,U,実績,13/05/08,↑↑↑↑‐,↑,↑,↑,↑,‐,22064192.0,1320888.0,1403649.0,962163.0,90
4,U,2014.03,13/05/08,NaN,NaN,NaN,NaN,NaN,NaN,23500000.0,1800000.0,1890000.0,1370000.0,－
5,U,NaN,13/08/02,↑↑↑↑‐,↑,↑,↑,↑,‐,24000000.0,1940000.0,2030000.0,1480000.0,－
6,U,NaN,13/11/06,↑↑↑↑‐,↑,↑,↑,↑,‐,25000000.0,2200000.0,2290000.0,1670000.0,－
7,U,NaN,14/02/04,↑↑↑↑‐,↑,↑,↑,↑,‐,25500000.0,2400000.0,2530000.0,1900000.0,－
8,U,実績,14/05/08,↑↓↓↓‐,↑,↓,↓,↓,‐,25691911.0,2292112.0,2441080.0,1823119.0,165
9,U,2015.03,14/05/08,NaN,NaN,NaN,NaN,NaN,NaN,25700000.0,2300000.0,2390000.0,1780000.0,－


In [156]:
tables[12][tables[12].index % 2 != 0]

,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,↓,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,↑,↑,↑,↑,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,↑,↓,↓,↓,‐,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
len(tables[12])

52

In [140]:
len(tables[12].columns)

14

In [109]:
# 後でhtml形式で読み込み可能なファイルとして書き出す方法

kessan_tables = open('kessan_tables.html', 'w')
kessan_tables.write(str(browser.select('table')))
kessan_tables.close()

tables = pd.read_html('/Users/Really/Stockyard/kessan_tables.html', header=0)

# テーブル内容の確認 (未ログイン)

In [6]:
url = 'https://kabutan.jp/stock/finance?code=1301&mode=k#kakosaikou_zisseki'

tables = get_table(url)

In [8]:
len(tables)

22

In [31]:
# ページ上部の主要指標情報 1-4の各項目と同じところっぽい
tables[0]

,"日経平均 (11:30) 22,404.85 +53.73 0.24%",日経平均 (11:30),"22,404.85",+53.73,0.24%,米ドル円 (11:58) 112.40 -0.84 -0.74%,米ドル円 (11:58),112.40,-0.84,-0.74%,"ＮＹダウ (終値) 23,458.36 +187.08 0.80%",ＮＹダウ (終値),"23,458.36",+187.08,0.80%,"上海総合 (終値) 3,399.87 -0.79 -0.02%",上海総合 (終値),"3,399.87",-0.79,-0.02%
0,日経平均 (11:30),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22404.85,53.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.24%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,米ドル円 (11:58),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,112.40,-0.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.74%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ＮＹダウ (終値),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23458.36,187.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.80%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,上海総合 (終値),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# 主要指標情報 日経平均
tables[1]

,日経平均 (11:30),Unnamed: 1
0,22404.85,53.73
1,0.24%,NaN


In [10]:
# 主要指標情報 米ドル円
tables[2]

,米ドル円 (11:58),Unnamed: 1
0,112.40,-0.84
1,-0.74%,NaN


In [11]:
# 主要指標情報 ＮＹダウ (終値)
tables[3]

,ＮＹダウ (終値),Unnamed: 1
0,23458.36,187.08
1,0.80%,NaN


In [12]:
# 主要指標情報 上海総合 (終値)
tables[4]

,上海総合 (終値),Unnamed: 1
0,3399.87,-0.79
1,-0.02%,NaN


In [13]:
# 検索窓
tables[5]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,「株探」で配信したニュース記事を検索します,Unnamed: 5


In [14]:
# 銘柄概要
tables[6]

,1301 極洋 東証１ 11:29,1301,極洋,東証１,11:29,Unnamed: 5
0,1301,極洋,東証１,11:29,NaN,NaN
1,"3,645円 前日比 -30 (-0.82%) 比較される銘柄： マルハニチロ 、日水 、カネコ種","3,645円",前日比,-30,(-0.82%),比較される銘柄： マルハニチロ 、日水 、カネコ種
2,"3,645円",前日比,-30,(-0.82%),NaN,NaN
3,比較される銘柄： マルハニチロ 、日水 、カネコ種,NaN,NaN,NaN,NaN,NaN


In [15]:
# 銘柄概要
tables[7]

,1301,極洋,東証１,11:29


In [16]:
# 銘柄概要
tables[8]

,"3,645円",前日比,-30,(-0.82%)
0,比較される銘柄： マルハニチロ 、日水 、カネコ種,NaN,NaN,NaN


In [17]:
# 銘柄概要
tables[9]

,業績：
0,水産・農林業
1,単位 100株


In [18]:
# ＰＥＲ ＰＢＲ 利回り 信用倍率
tables[10]

,ＰＥＲ,ＰＢＲ,利回り,信用倍率
0,14.2倍,1.46倍,1.37％,1.80倍


In [34]:
# 比較参照用
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目（売上高欄に掲載）', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開又は更新日'],
      dtype='object')

In [19]:
# 通期業績
tables[11]
# 株プロに無い項目: １株配

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014.03,202387.0,2915.0,2985.0,2968.0,28.3,5.0,14/05/09
2,2015.03,218350.0,2460.0,2107.0,2433.0,23.2,5.0,15/05/08
3,2016.03,226626.0,2433.0,2814.0,1799.0,17.1,5.0,16/05/09
4,2017.03,236561.0,3723.0,3709.0,2422.0,230.7,60.0,17/05/11
5,予 2018.03,250000.0,4000.0,4000.0,2700.0,257.0,50.0,17/05/11
6,前期比,5.7,7.4,7.8,11.5,11.4,NaN,(%)
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# 業績予想修正履歴
tables[12]

,決算期,修正日,修正方向,売上高,営業益,経常益,最終益,修正配当,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,2013.03,12/05/11,NaN,NaN,NaN,NaN,NaN,NaN,185000.0,3200.0,3100.0,1800.0,50.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,12/10/26,↓↓↓↓→,↓,↓,↓,↓,→,175000.0,2000.0,2200.0,1100.0,50.0
3,↓,↓,↓,↓,→,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,実績,13/05/10,↑↑↑↑→,↑,↑,↑,↑,→,178046.0,2324.0,2262.0,1269.0,50.0
5,↑,↑,↑,↑,→,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,2014.03,13/05/10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,13/10/21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,実績,14/05/09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,2015.03,14/05/09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# 不明
tables[13]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4


In [22]:
# 修正履歴修正方向
tables[14]

,↓,↓.1,↓.2,↓.3,→


In [23]:
# 修正履歴修正方向
tables[15]

,↑,↑.1,↑.2,↑.3,→


In [24]:
# 過去最高 【実績】
tables[16]

,Unnamed: 0,売上高,営業益,経常益,最終益,１株益
0,過去最高,236561.00,3723.00,3709.00,5540.0,230.70
1,決算期,2017.03,2017.03,2017.03,1985.1,2017.03


In [25]:
# 下期業績
tables[17]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,14.10-03,115874.0,1739.0,1421.0,168.0,1.6,5.0,15/05/08
1,15.10-03,118127.0,1307.0,1407.0,888.0,8.5,5.0,16/05/09
2,16.10-03,126991.0,2552.0,2705.0,1236.0,117.7,60.0,17/05/11
3,予 17.10-03,129542.0,1754.0,1604.0,1067.0,101.6,50.0,17/11/06
4,前年同期比,2.0,-31.3,-40.7,-13.7,-13.7,NaN,(%)


In [26]:
# 第２四半期累計決算【実績】
tables[18]

,決算期,売上高,営業益,経常益,最終益,１株益,対通期進捗率,発表日
0,15.04-09,108499.0,1126.0,1407,911.0,8.7,50.0,15/11/06
1,16.04-09,109570.0,1171.0,1004,1186.0,113.0,27.1,16/11/04
2,17.04-09,120458.0,2246.0,2396,1633.0,155.5,59.9,17/11/06
3,前年同期比,9.9,91.8,2.4倍,37.7,37.6,NaN,(%)


In [27]:
# ３ヵ月業績の推移【実績】
# 累積ではなく差分？
tables[19]

,決算期,売上高,営業益,経常益,最終益,１株益,売上営業損益率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15.10-12,70391.0,1341.0,1281,222.0,2.1,1.9,16/02/05
2,16.01-03,47736.0,-34.0,126,666.0,6.3,-0.1,16/05/09
3,16.04-06,52206.0,467.0,380,551.0,5.2,0.9,16/08/05
4,16.07-09,57364.0,704.0,624,635.0,60.5,1.2,16/11/04
5,16.10-12,70405.0,1701.0,1823,1263.0,120.3,2.4,17/02/10
6,17.01-03,56586.0,851.0,882,-27.0,-2.6,1.5,17/05/11
7,17.04-06,56844.0,979.0,1103,754.0,71.8,1.7,17/08/04
8,17.07-09,63614.0,1267.0,1293,879.0,83.7,2.0,17/11/06
9,前年同期比,10.9,80.0,2.1倍,38.4,38.3,NaN,(%)


In [28]:
# 財務 【実績】
tables[20]
# 株プロに無い項目: 自己資本比率, 自己資本, 剰余金, 有利子負債倍率

,決算期,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率,発表日
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015.03,215.65,25.5,88937.0,22649.0,16537.0,2.09,15/05/08
2,2016.03,214.97,23.9,94608.0,22578.0,17739.0,2.24,16/05/09
3,2017.03,2378.09,25.6,97391.0,24976.0,19637.0,2.04,17/05/11
4,17.04-09,－,22.0,119806.0,26312.0,20640.0,2.50,17/11/06
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 日経平均チャート切り替え
tables[21]

,Unnamed: 0,日中足
0,日 足,NaN
